In [1]:
import requests
import json
from datetime import datetime

class SimpleContextTree:
    def __init__(self):
        self.states = {}  # state_id -> state_data
        self.current_state = None
        self.counter = 0
        
    def save_state(self, context, message, response, parent_id=None):
        self.counter += 1
        state_id = f"state_{self.counter}"
        
        self.states[state_id] = {
            'id': state_id,
            'parent_id': parent_id,
            'context': context,
            'message': message,
            'response': response,
            'timestamp': datetime.now().isoformat(),
            'children': []
        }
        
        # Add to parent's children list
        if parent_id and parent_id in self.states:
            self.states[parent_id]['children'].append(state_id)
            
        self.current_state = state_id
        return state_id
    
    def get_state(self, state_id):
        return self.states.get(state_id)
    
    def print_tree(self, state_id=None, indent=0):
        if state_id is None:
            # Find root nodes (no parent)
            roots = [s for s in self.states.values() if s['parent_id'] is None]
            for root in roots:
                self.print_tree(root['id'])
            return
            
        state = self.states[state_id]
        prefix = "  " * indent + ("└─ " if indent > 0 else "")
        print(f"{prefix}{state['id']}: {state['message'][:50]}...")
        
        for child_id in state['children']:
            self.print_tree(child_id, indent + 1)

def chat_with_ollama(message, model="llama3.2", context=None):
    """Simple function to chat with Ollama"""
    url = "http://localhost:11434/api/generate"
    
    payload = {
        "model": model,
        "prompt": message,
        "stream": False
    }
    
    if context:
        payload["context"] = context
    
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            result = response.json()
            return {
                "response": result.get("response", ""),
                "context": result.get("context", [])
            }
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"Connection error: {e}")
        return None

# Test the context tree
def test_context_tree():
    tree = SimpleContextTree()
    
    print("=== Testing Context Tree ===\n")
    
    # First interaction
    print("1. First message...")
    result = chat_with_ollama("What's the capital of France?")
    if result:
        state1 = tree.save_state(
            result["context"], 
            "What's the capital of France?", 
            result["response"]
        )
        print(f"Response: {result['response'][:100]}...")
        print(f"Saved as: {state1}\n")
    
    # Second interaction (continues from first)
    print("2. Follow-up message...")
    result = chat_with_ollama("What's its population?", context=tree.get_state(state1)["context"])
    if result:
        state2 = tree.save_state(
            result["context"],
            "What's its population?",
            result["response"],
            parent_id=state1
        )
        print(f"Response: {result['response'][:100]}...")
        print(f"Saved as: {state2}\n")
    
    # Branch from first state
    print("3. Branching from first state...")
    first_state_context = tree.get_state(state1)["context"]
    result = chat_with_ollama("What about the capital of Germany?", context=first_state_context)
    if result:
        state3 = tree.save_state(
            result["context"],
            "What about the capital of Germany?",
            result["response"],
            parent_id=state1
        )
        print(f"Response: {result['response'][:100]}...")
        print(f"Saved as: {state3}\n")
    
    # Show the tree structure
    print("=== Conversation Tree ===")
    tree.print_tree()
    
    print(f"\nTotal states: {len(tree.states)}")
    print("State details:")
    for state_id, state in tree.states.items():
        print(f"  {state_id}: parent={state['parent_id']}, children={state['children']}")

if __name__ == "__main__":
    test_context_tree()

ModuleNotFoundError: No module named 'requests'